# Data Preprocessing Tools

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [2]:
dataset = pd.read_csv('Data.csv')

# import all rows and all but the last column
# matric of features is the data that will determine the result
matrix_of_features = dataset.iloc[:, :-1].values

# import all rows from the last column
# dependant variable vector is the result we are calculating
dependant_variable_vector = dataset.iloc[:, -1].values

In [3]:
print(matrix_of_features)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [4]:
print(dependant_variable_vector)

['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


## Taking care of missing data

In [5]:
# < 1% of data missing, then removing the missing entries is acceptable.
# Otherwise, commonly accepted to replace missing data with the average of the rest of the data in that column
from sklearn.impute import SimpleImputer

# what value is being replaced and with what?
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

#'fit' the features to the imputer (just the numerical columns)
imputer.fit(matrix_of_features[:, 1:3])

# make the replacement and update the numerical columns of the original matrix
matrix_of_features[:, 1:3] = imputer.transform(matrix_of_features[:, 1:3])

In [6]:
print(matrix_of_features)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## Encoding categorical data

### Encoding the Independent Variable

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# prepare the columntransformer
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')

# fit and transform the column using OneHotEncoder
# we use 1HE because it converts each unique string into a numerical value but
# doesn't assign a numerical ORDER due to the use of columns
# This is important for ML models later
matrix_of_features = ct.fit_transform(matrix_of_features)

# convert back into numpy array
matrix_of_features = np.array(matrix_of_features)

In [8]:
print(matrix_of_features)

[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


### Encoding the Dependent Variable

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dependant_variable_vector = le.fit_transform(dependant_variable_vector)
# future models don't need this to be numpy array

In [10]:
print(dependant_variable_vector)

[0 1 0 0 1 1 0 1 0 1]


## Splitting the dataset into the Training set and Test set

In [11]:
from sklearn.model_selection import train_test_split

# split data into training and test data at a 80/20 ratio
x_train, x_test, y_train, y_test = train_test_split(matrix_of_features, dependant_variable_vector, test_size = 0.2, random_state = 1)

In [12]:
print(x_train)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]


In [13]:
print(x_test)

[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


In [14]:
print(y_train)

[0 1 0 0 1 1 0 1]


In [15]:
print(y_test)

[0 1]


## Feature Scaling

In [16]:
# Standardisation puts all feature values between -3 and +3
# Normalisation putss all features between 0 and 1

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

# you don't need to apply feature scaling to 1HE dummy variables, only to numerical values
x_train[:, 3:] = sc.fit_transform(x_train[:, 3:])

# only apply 'transform' to the test data as we want to use the same Scaler as the Training data
x_test[:, 3:] = sc.transform(x_test[:, 3:])

In [17]:
print(x_train)

[[0.0 0.0 1.0 -0.19159184384578545 -1.0781259408412425]
 [0.0 1.0 0.0 -0.014117293757057777 -0.07013167641635372]
 [1.0 0.0 0.0 0.566708506533324 0.633562432710455]
 [0.0 0.0 1.0 -0.30453019390224867 -0.30786617274297867]
 [0.0 0.0 1.0 -1.9018011447007988 -1.420463615551582]
 [1.0 0.0 0.0 1.1475343068237058 1.232653363453549]
 [0.0 1.0 0.0 1.4379472069688968 1.5749910381638885]
 [1.0 0.0 0.0 -0.7401495441200351 -0.5646194287757332]]


In [18]:
print(x_test)

[[0.0 1.0 0.0 -1.4661817944830124 -0.9069571034860727]
 [1.0 0.0 0.0 -0.44973664397484414 0.2056403393225306]]
